In [92]:
# !pip install arxiv

In [93]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [94]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200,lang="en")
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [95]:
wiki_tool.name

'wikipedia'

In [96]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [97]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
# https://www.anthropic.com/engineering/building-effective-agents

loader=WebBaseLoader("https://www.anthropic.com/engineering/building-effective-agents")
docs=loader.load()

documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents=documents,embedding=OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x169668910>, search_kwargs={})

In [107]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"Building_Effective_Agents_Search","Information_about_Agents")
retriever_tool.name

'Building_Effective_Agents_Search'

In [108]:
## Arxiv Tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200,lang="en")
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [109]:
tools=[wiki_tool,arxiv_tool,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/chaithanyamanasreddymatta/Documents/Learning/AI/rag-langchain/env/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Building_Effective_Agents_Search', description='Information_about_Agents', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10d247a30>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.

In [110]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [111]:
### Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [112]:
## Agent Executer

from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [117]:
# agent_executer.invoke({"input":"Tell me about Avatar movie"})
agent_executer.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper titled "Heat-bath random walks with Markov bases" (arXiv:1605.08386) was published on May 26, 2016, by authors Caprice Stanley and Tobias Windisch. 

The paper studies graphs on lattice points where the edges are derived from a finite set of allowed edges. It focuses on the concept of heat-bath random walks and their relationship with Markov bases, which are used in the context of statistical models and combinatorial structures. 

If you need more specific details or insights from the paper, feel free to ask!

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper titled "Heat-bath random walks with Markov bases" (arXiv:1605.08386) was published on May 26, 2016, by authors Caprice Stanley and Tobias Windisch. \n\nThe paper studies graphs on lattice points where the edges are derived from a finite set of allowed edges. It focuses on the concept of heat-bath random walks and their relationship with Markov bases, which are used in the context of statistical models and combinatorial structures. \n\nIf you need more specific details or insights from the paper, feel free to ask!'}